In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# data columns

# videoId,
# title,
# description,
# tags,
# viewCount,
# likeCount,
# favoriteCount,
# commentCount,
# publishedAt,
# channelId,
# channelTitle,
# country

use_columns = ['videoId', 'title', 'description', 'tags', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount', 'publishedAt', 'channelId', 'channelTitle', 'country']


In [ ]:
from dotenv import load_dotenv
import os
import sys
import pandas as pd
import numpy as np
from youtubeapi import YoutubeApi
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.font_manager as fm
# sys.path.append('../refs/')
# from utils import Utils
from utils import utils

# font setting
font_location = 'C:/Windows/Fonts/Malgun.ttf' if sys.platform == 'win32' else '/System/Library/Fonts/AppleSDGothicNeo.ttc'
font_name = fm.FontProperties(fname=font_location).get_name()
# print(font_name)
matplotlib.rc('font', family=font_name)

load_dotenv()
apiKey = os.environ.get('YoutubeApiToken')
url = 'https://youtube.googleapis.com/youtube/v3/'

api = YoutubeApi(apiKey, use_columns)

In [ ]:
asia_data = pd.read_csv('../data/asia_data_kor.csv')
asia_data['Year'].fillna(method='ffill', inplace=True)
# asia_data

In [ ]:
# 코로나 이전 기준 2019년 11월
before_start_date = '2018년12월'
before_end_date = '2019년11월'

# 코로나 이후 기준 2023년 5월 (데이터가 수집된 마지막 기준) 부터 1년
after_start_date = '2022년6월'
after_end_date = '2023년5월'

# 국가
countries = ['일본', '베트남', '대만', '홍콩', '필리핀', '태국']

In [ ]:
asia_data['date'] = asia_data['Year'] + asia_data['month']
asia_data

In [ ]:
results = pd.DataFrame()

# 각 국가별 loop
for country in countries:
  # 코로나 이전 (2018년12월 ~ 2019년11월)
  before = utils.get_dataframe_by_period2(asia_data, before_start_date, before_end_date, country)
  # 코로나 이후 (2022년6월 ~ 2023년5월)
  after = utils.get_dataframe_by_period2(asia_data, after_start_date, after_end_date, country)
  
  temp1 = before.loc[before['count'].idxmax()].to_frame().T
  temp1 = temp1.reset_index(drop=False)
  temp1 = temp1.rename({'index': 'before_max_index', 'count': 'before_max_count', 'date': 'before_max_date'}, axis=1)
  
  temp2 = after.loc[after['count'].idxmax()].to_frame().T
  temp2 = temp2.reset_index(drop=False)
  temp2 = temp2.rename({'index': 'after_max_index', 'count': 'after_max_count', 'date': 'after_max_date'}, axis=1)
  
  temp3 = pd.Series(country).to_frame().T
  
  result = pd.concat([temp3, temp1, temp2], axis=1)
  result = result.rename({0: 'country'}, axis=1)
  result
  
  results = pd.concat([results, result])

results.to_csv('../data/countries_max.csv', encoding='utf-8-sig')


In [ ]:
# 코로나 이전 (2018년12월 ~ 2019년11월)
before = utils.get_dataframe_by_period2(asia_data, before_start_date, before_end_date, countries[0])
# display(before)

# 코로나 이후 (2022년6월 ~ 2023년5월)
after = utils.get_dataframe_by_period2(asia_data, after_start_date, after_end_date, countries[0])
# display(after)

In [ ]:
temp1 = before.loc[before['count'].idxmax()].to_frame().T
temp1 = temp1.reset_index(drop=False)
temp1 = temp1.rename({'index': 'before_max_index', 'count': 'before_max_count', 'date': 'before_max_date'}, axis=1)
temp1

In [ ]:
temp2 = after.loc[after['count'].idxmax()].to_frame().T
temp2 = temp2.reset_index(drop=False)
temp2 = temp2.rename({'index': 'after_max_index', 'count': 'after_max_count', 'date': 'after_max_date'}, axis=1)
temp2

In [ ]:
temp3 = pd.Series('일본').to_frame().T
temp3

In [ ]:
result = pd.concat([temp3, temp1, temp2], axis=1)
result = result.rename({0: 'country'}, axis=1)
result

In [ ]:
xxtemp = before.loc[before['count'].idxmax()]
xxtemp = xxtemp.rename({'count': 'before_max_count', 'date': 'before_max_date'})
xxtemp = xxtemp.rename('index')
xxtemp

In [ ]:
temp1 = before.loc[before['count'].idxmax()]
temp1 = temp1.rename({'count': 'before_max_count', 'date': 'before_max_date'})
temp1 = temp1.reset_index(name='values')
display(temp1)

temp2 = after.loc[after['count'].idxmax()]
temp2 = temp2.rename({'count': 'after_max_count', 'date': 'after_max_date'})
temp2 = temp2.reset_index(name='values')
display(temp2)

temp3 = pd.Series('일본')
temp3.reset_index(name='values')
display(temp3)

pd.concat([temp3, temp1, temp2], axis=1)

In [ ]:
results = []

results.append(pd.concat([temp3, temp1, temp2], axis=1))

results = pd.DataFrame(results)
results